In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import pandas as pd
import keras
import os
import time

Using TensorFlow backend.


In [2]:
validation_csv = pd.read_csv('/Volumes/Mac/BepDataResNet/ManuallyAnotatedFileList/validation.csv', sep = ',', names = ["File_path",'Face_x', 'Face-y', "Face_width", "Face_height",
                                                                   'Facial_landmarks', 'Expression', 'Valence', 'Arousal'], index_col = "File_path")

training_csv = pd.read_csv('/Volumes/Mac/BepDataResNet/ManuallyAnotatedFileList/training.csv', sep = ',', names = ["File_path",'Face_x', 'Face-y', "Face_width", "Face_height",
                                                                   'Facial_landmarks', 'Expression', 'Valence', 'Arousal'], index_col= "File_path", header = 1)

training_csv['Expression'] = pd.to_numeric(training_csv['Expression'])
validation_csv['Expression'] = pd.to_numeric(validation_csv['Expression'])

In [3]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

#found_image_paths = getListOfFiles("/Volumes/Mac/BepDataResNet/ManuallyAnnotated")
#train_and_val = pd.concat([training_csv, validation_csv])

In [4]:
def read_data(input_dir = "/Volumes/Mac/BepDataResNet/ManuallyAnnotated",
                   output_dir = "/Users/blazejmanczak/Desktop"):
    
    start = time.time()
    
    #"""
    validation_csv = pd.read_csv('/Volumes/Mac/BepDataResNet/ManuallyAnotatedFileList/validation.csv', sep = ',', names = ["File_path",'Face_x', 'Face-y', "Face_width", "Face_height",
                                                                   'Facial_landmarks', 'Expression', 'Valence', 'Arousal'], index_col = "File_path")

    training_csv = pd.read_csv('/Volumes/Mac/BepDataResNet/ManuallyAnotatedFileList/training.csv', sep = ',', names = ["File_path",'Face_x', 'Face-y', "Face_width", "Face_height",
                                                                   'Facial_landmarks', 'Expression', 'Valence', 'Arousal'], index_col= "File_path", header = 1)

    #"""
    training_csv['Expression'] = pd.to_numeric(training_csv['Expression'])
    validation_csv['Expression'] = pd.to_numeric(validation_csv['Expression'])
    
    train_and_val = pd.concat([training_csv, validation_csv])
    train_and_val['RowNumber'] = range(1, train_and_val.shape[0]+1)

    found_image_paths = getListOfFiles(input_dir)
    
    X_train = []
    Y_train = []
    
    X_test = []
    Y_test = []

    failures = 0 # keeping track of failed imports
    class_count = dict(zip(range(0,11), [0]*11))# keep track of label counts

    for path in found_image_paths[1:200]:

        try: # image is avaliable
            
            path_for_csv =  path.split("Manually_Annotated_Images")[1][1:]
            label_and_rowNum = train_and_val.loc[path_for_csv, ["Expression", "RowNumber"]]
            
            img = load_img(path, target_size = (256,256))
            x = img_to_array(img)

            if label_and_rowNum[1] > training_csv.shape[0]:  # faster way of establishing whether row comes from validation set
                X_test.append(x)
                Y_test.append(label_and_rowNum[0])
            else:
                X_train.append(x)
                Y_train.append(label_and_rowNum[0])
                
            class_count[label_and_rowNum[0]] += 1 # keep track of label counts

        except: 
            failures += 1

    
    X_train = np.asarray(X_train).astype('float16') / 255. # scaling the images by 255
    Y_train = keras.utils.to_categorical(Y_train, num_classes = 11)
    
    X_test = np.asarray(X_test).astype('float16') / 255. # scaling the images by 255
    Y_test = keras.utils.to_categorical(Y_test, num_classes = 11)
  
    np.savez_compressed("/Volumes/Mac/BepDataResNet/trainData", X_train = X_train, Y_train = Y_train)
    np.savez_compressed("/Volumes/Mac/BepDataResNet/testData", X_test = X_test, Y_test = Y_test)
    
    sum_all_vals =  sum(class_count.values())
    for key in class_count.keys():
        class_count[key] = round(class_count[key] / sum_all_vals), 2)
        
    print ("Test Data size", X_test.shape, Y_test.shape)
    print ("Train Data size", X_train.shape, Y_train.shape)
    print("There have been {} failures".format(failures))
    print("The class distirbution is:", class_count)
    print("The function took", round(time.time() - start,2), "seconds")
    

read_data()
    

Test Data size (3, 256, 256, 3) (3, 11)
Train Data size (194, 256, 256, 3) (194, 11)
There have been 2 failures
The class distirbution is: {0: 0.15, 1: 0.51, 2: 0.25, 3: 0.1, 4: 0.05, 5: 0.0, 6: 0.17, 7: 0.07, 8: 0.29, 9: 0.27, 10: 0.91}
The function took 70.46 seconds
